In [1]:
#Library installations
import numpy as np

from qiskit import QuantumCircuit, transpile
from qiskit.visualization.counts_visualization import plot_histogram
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qbraid.runtime.qiskit import QiskitRuntimeProvider
from qbraid.runtime.qiskit.job import QiskitJob

from qiskit_ibm_runtime import QiskitRuntimeService

from dotenv import load_dotenv

import os

In [3]:
provider = QiskitRuntimeService(channel='ibm_cloud', token='qNhbkxsOZX2nmXx_i6gnj4lzRg84fuBJ37J6pO4m72tP', instance='crn:v1:bluemix:public:quantum-computing:us-east:a/de667816234545beb326942f831d531e:2f94aa43-a60b-4f7f-831f-8964ba3251c7::')
qbraid_provider = QiskitRuntimeProvider(runtime_service=provider)
qbraid_provider.get_devices()

[<qbraid.runtime.qiskit.device.QiskitBackend('ibm_brisbane')>,
 <qbraid.runtime.qiskit.device.QiskitBackend('ibm_fez')>,
 <qbraid.runtime.qiskit.device.QiskitBackend('ibm_kyoto')>,
 <qbraid.runtime.qiskit.device.QiskitBackend('ibm_osaka')>,
 <qbraid.runtime.qiskit.device.QiskitBackend('ibm_sherbrooke')>,
 <qbraid.runtime.qiskit.device.QiskitBackend('ibm_torino')>,
 <qbraid.runtime.qiskit.device.QiskitBackend('ibmq_qasm_simulator')>,
 <qbraid.runtime.qiskit.device.QiskitBackend('simulator_stabilizer')>,
 <qbraid.runtime.qiskit.device.QiskitBackend('simulator_mps')>,
 <qbraid.runtime.qiskit.device.QiskitBackend('simulator_extended_stabilizer')>,
 <qbraid.runtime.qiskit.device.QiskitBackend('simulator_statevector')>]

In [11]:
#Building a 100 qubit quantum circuit with all qubits in equal superposition using hadamard gate
circ = QuantumCircuit(1)
for i in range(1):
    circ.h(i)
circ.measure_all()
circ.draw()

┌───┐ ░ ┌─┐
     q: ┤ H ├─░─┤M├
        └───┘ ░ └╥┘
meas: 1/═════════╩═
                 0

In [9]:
def launch_job(qbraid_provider, device_name, shot_count, circ):
    #load_dotenv()
    #API_KEY = os.getenv('IBM_APIKEY')
    #provider = QiskitRuntimeProvider(API_KEY)
    provider = qbraid_provider
    device = provider.get_device(device_name)
    pm = generate_preset_pass_manager(optimization_level=1, target=device._backend.target)
    transpiled_circuit = pm.run(circ)
    job = device.run(transpiled_circuit, shots=shot_count, memory=True)
    print(job._job_id)


def get_job_results(job_id, length):
    API_KEY = os.getenv('IBM_APIKEY')
    provider = QiskitRuntimeService('ibm_quantum', API_KEY)
    job = provider.job(job_id)
    QBraidJob = QiskitJob(job_id, job)
    result = QBraidJob.result()

    rawData = result.measurements()

    unbrokenData = []
    for shot in rawData:
        for meas in shot:
            unbrokenData.append(meas)

    data = []
    for i in range(0, len(unbrokenData), length):
        data.append(''.join(map(str, unbrokenData[i:i+length])))
    
    return data

def write_data_to_file(data, file_path, labels):
    with open(file_path, 'a') as file:
        labelString = ''
        for label in labels:
            labelString += (label + ' ')
        labelString = labelString[:-1]
        for output in data:
            file.write(output + ' ' + labelString + '\n')
    file.close()

In [10]:
launch_job(qbraid_provider, 'ibm_fez', 1, circ)

cqtrrgkthjpt4ifqodpg


In [19]:
launch_job('ibm_brisbane', 10000)

ctmnhb54gjh0008pcke0


In [21]:
launch_job('ibm_kyoto', 10000)

ctmnhr7sj17g0081x3zg


In [22]:
launch_job('ibm_osaka', 10000)

ctmnhwfsj17g0081x400


In [23]:
launch_job('ibm_sherbrooke', 10000)

ctmnhyzsj17g0081x40g


In [24]:
write_data_to_file(get_job_results('ctmnhb54gjh0008pcke0'), 'data/IBM_Superposition_QRNG_100qubit.txt', ['quantum', 'ibm_brisbane'])

In [25]:
write_data_to_file(get_job_results('ctmnhr7sj17g0081x3zg'), 'data/IBM_Superposition_QRNG_100qubit.txt', ['quantum', 'ibm_kyoto'])

In [26]:
write_data_to_file(get_job_results('ctmnhwfsj17g0081x400'), 'data/IBM_Superposition_QRNG_100qubit.txt', ['quantum', 'ibm_osaka'])

In [27]:
write_data_to_file(get_job_results('ctmnhyzsj17g0081x40g'), 'data/IBM_Superposition_QRNG_100qubit.txt', ['quantum', 'ibm_sherbrooke'])

In [ ]:
#Creating euql amount of non-quantum training data using numpy random module
with open('data/IBM_Superposition_QRNG_100qubit.txt', 'a') as file:
    file.write('\n')
    rng = np.random.default_rng()
    for i in range(40000):
        prng = rng.integers(0, 2, 100)
        rngString = ''.join(str(x) for x in prng)
        output = rngString + ' non-quantum N/A\n'
        file.write(output)
    
file.close()

Refactoring QRNG generation code for 10 qubit circuit

In [5]:
circ = QuantumCircuit(10)
for i in range(10):
    circ.h(i)
circ.measure_all()

In [6]:
launch_job('ibm_brisbane', 100000, circ)

ctw09env0kkg008pzcpg


In [7]:
launch_job('ibm_kyoto', 100000, circ)

ctw0a0r46w90008rdt2g


In [8]:
launch_job('ibm_osaka', 100000, circ)

ctw0a4rv0kkg008pzcq0


In [9]:
launch_job('ibm_sherbrooke', 100000, circ)

/Users/shrey/Dev/qc-classifier-experiments/.venv/lib/python3.12/site-packages/qbraid/runtime/device.py:135: UserWarning: Device is not online. Depending on the provider queueing system, submitting this job may result in an exception or a long wait time.
  warnings.warn(
/Users/shrey/Dev/qc-classifier-experiments/.venv/lib/python3.12/site-packages/qiskit_ibm_runtime/ibm_backend.py:711: UserWarning: The backend ibm_sherbrooke is currently paused.
  warnings.warn(f"The backend {self.name} is currently paused.")


IBMBackendApiError: 'Error submitting job: \'409 Client Error: Conflict for url: https://api.quantum.ibm.com/runtime/jobs. {"errors":[{"message":"You have reached the limit of 3 pending  jobs. Please wait for a job to complete or cancel one before submitting anything new.","code":3458,"solution":"Wait until some previous jobs were finished. You can cancel pending jobs to run new jobs.","more_info":"https://docs.quantum-computing.ibm.com/errors"}]}\''

In [10]:
write_data_to_file(get_job_results('ctw0a0r46w90008rdt2g', 10), 'data/IBM_Superposition_QRNG_10qubit.txt', ['quantum', 'ibm_kyoto'])

In [11]:
write_data_to_file(get_job_results('ctw0a4rv0kkg008pzcq0', 10), 'data/IBM_Superposition_QRNG_10qubit.txt', ['quantum', 'ibm_osaka'])

In [12]:
write_data_to_file(get_job_results('ctw09env0kkg008pzcpg', 10), 'data/IBM_Superposition_QRNG_10qubit.txt', ['quantum', 'ibm_brisbane'])